In [9]:
import os

from glob import glob

def collect_class_forest(path):
    pyfiles = list(glob(path+"/**/*.py", recursive=True))
#     print("# of python files:", len(pyfiles))
#     print(pyfiles)
    

In [10]:
def test_class_forest():
    expected = {
        "First": {
          "Second": {
              "Third": {}
          }
        },
        "A": {
          "B": {},
          "D": {}
        }
    }

    result = collect_class_forest("./src")
    assert expected == result

In [11]:
test_class_forest()

['./src/example.py']


AssertionError: 

In [12]:
import ast

def getAST(path: str):
    with open(path, "r") as f:
        source = f.read()
        
    return ast.parse(source)

In [13]:
from ast import NodeVisitor

class ClassVisitor(NodeVisitor):
    def __init__(self):
        super().__init__()
        self.gen_dict = {}
        
    def visit_ClassDef(self, node):
        parents = []
        for base in node.bases:
            if isinstance(base, ast.Attribute):
                parents.append(base.attr)
            elif isinstance(base, ast.Subscript):
                parents.append(base.value.id)
            elif isinstance(base, ast.Name):
                parents.append(base.id)
            else:
                raise Exception(f"Unknown base type: {type(base)}")
        self.gen_dict[node.name] = { "parents": parents }
        return super().generic_visit(node)

In [14]:
def collect_class_forest(path):
    pyfiles = list(glob(path+"/**/*.py", recursive=True))
    
    myvisitor = ClassVisitor()
    for pfile in pyfiles:
        myast = getAST(pfile)
        myvisitor.visit(myast)
    
    print(myvisitor.gen_dict)

In [15]:
test_class_forest()

{'First': {'parents': []}, 'Second': {'parents': ['First']}, 'Third': {'parents': ['Second']}, 'A': {'parents': []}, 'B': {'parents': ['A']}, 'C': {'parents': []}, 'D': {'parents': ['A']}}


AssertionError: 

In [18]:
def collect_class_forest(path):
    pyfiles = list(glob(path+"/**/*.py", recursive=True))
    
    myvisitor = ClassVisitor()
    for pfile in pyfiles:
        myast = getAST(pfile)
        myvisitor.visit(myast)
    
    forwardmap = {}
    
    for key, values in myvisitor.gen_dict.items():
        for parent in values["parents"]:
            if parent in forwardmap.keys():
                forwardmap[parent][key] = {}
            else:
                forwardmap[parent] = {key:{}}
    
    print(forwardmap)

In [19]:
test_class_forest()

{'First': {'Second': {}}, 'Second': {'Third': {}}, 'A': {'B': {}, 'D': {}}}


AssertionError: 

In [24]:
def collect_class_forest(path):
    pyfiles = list(glob(path+"/**/*.py", recursive=True))
    
    myvisitor = ClassVisitor()
    for pfile in pyfiles:
        myast = getAST(pfile)
        myvisitor.visit(myast)
    
    forwardmap = {}
    
    for key, values in myvisitor.gen_dict.items():
        for parent in values["parents"]:
            if parent in forwardmap.keys():
                forwardmap[parent][key] = {}
            else:
                forwardmap[parent] = {key:{}}
    
    mystack = []
    
    forest = {}
    for aroot, values in forwardmap.items():
        for subclass, adict in values.items():
            mystack.append( (aroot, subclass) )
            
    while not mystack.empty():
        sup, sub = mystack.peek()
        if sub in forwardmap.keys():
            

In [25]:
test_class_forest()

[('First', 'Second'), ('Second', 'Third'), ('A', 'B'), ('A', 'D')]


AssertionError: 

In [40]:
def collect_class_forest(path):
    pyfiles = list(glob(path+"/**/*.py", recursive=True))
    
    myvisitor = ClassVisitor()
    for pfile in pyfiles:
        myast = getAST(pfile)
        myvisitor.visit(myast)
    
    forwardmap = {}
    
    for key, values in myvisitor.gen_dict.items():
        for parent in values["parents"]:
            if parent in forwardmap.keys():
                forwardmap[parent][key] = {}
            else:
                forwardmap[parent] = {key:{}}
                
                
    def update_children(p, fmap):
        if p not in fmap.keys():
            return {}
        else:
            amap = {}
            for child in fmap[p]:
                children = update_children(child, fmap)
                amap[child] = children   
            return amap
        
    forest = {}
    for key, children in forwardmap.items():
        if key in myvisitor.gen_dict.keys() and len(myvisitor.gen_dict[key]["parents"]) == 0:
#             for child in children.keys():
            forest[key] = update_children(key, forwardmap)
    
    return forest

In [36]:
test_class_forest()

In [41]:
collect_class_forest("./cli")

{'SessionTestBase': {'TestSessionFlow': {}, 'TestSession': {}},
 'CookieTestBase': {'TestExpiredCookies': {}, 'TestCookieStorage': {}},
 'Environment': {'MockEnvironment': {'PersistentMockEnvironment': {}},
  'HTTPieEnvironment': {},
  'LocalCommandEnvironment': {}},
 'BaseCLIResponse': {'BytesCLIResponse': {}, 'StrCLIResponse': {}},
 'HTTPMessage': {'HTTPResponse': {}, 'HTTPRequest': {}},
 'ChunkedStream': {'ChunkedUploadStream': {},
  'ChunkedMultipartUploadStream': {}},
 'BasePlugin': {'AuthPlugin': {'BuiltinAuthPlugin': {'BasicAuthPlugin': {},
    'DigestAuthPlugin': {},
    'BearerAuthPlugin': {}}},
  'TransportPlugin': {'FakeTransportPlugin': {}},
  'ConverterPlugin': {'SortJSONConverterPlugin': {}},
  'FormatterPlugin': {'XMLFormatter': {},
   'HeadersFormatter': {},
   'JSONFormatter': {},
   'ColorFormatter': {}}},
 'BaseStream': {'RawStream': {},
  'EncodedStream': {'PrettyStream': {'BufferedPrettyStream': {}}}},
 'BaseDisplay': {'DummyDisplay': {},
  'StatusDisplay': {},
  '